In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/cnews_train.txt


In [2]:
import torch
from torch import nn  #nn.Module是PyTorch提供的神经网络类，并在类中实现了网络各层的定义及前向计算与反向传播机制
import torch.nn.functional as F   #包含 torch.nn 库中所有函数同时包含大量 loss 和 activation function
from tqdm.notebook import tqdm   #tqdm中的tqdm()是实现进度条美化的基本方法
from sklearn.metrics import classification_report   #显示分类指标
from transformers import BertTokenizer  #BertTokenizer分词器
from transformers import BertModel  #对象构建
from torch.utils.data import TensorDataset #TensorDataset(tensor1,tensor2)的功能就是形成数据tensor1和标签tensor2的对应


In [3]:
# 原始数据
df = pd.read_table('/kaggle/input/dataset/cnews_train.txt',header=None)
columns = ['category','text']
df.columns = columns

In [4]:
df.dropna(axis=0, how='any', inplace=True) #drop掉所有可能出现nan的行并覆盖原有的df
df[df.isnull().T.any().T]  #找出含有nan的所有行
df.reset_index(drop=True,inplace=True) #重置索引值并覆盖原有的df

In [5]:
df.category.value_counts() #统计数类别个数

体育    5000
娱乐    5000
家居    5000
房产    5000
教育    5000
时尚    5000
时政    5000
游戏    5000
科技    5000
财经    5000
Name: category, dtype: int64

In [6]:
possible_labels = df.category.unique() #获取不重复的唯一的category
possible_labels #查看

array(['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经'],
      dtype=object)

In [7]:
#标签对应相应的数字
label_dict = {}  #定义一个新的字典
for index, possible_label in enumerate(possible_labels):  #循环查看possible_labels以及对应的索引值
    label_dict[possible_label] = index #让字典中的key（possible_label）与其索引值一一对应
label_dict   #查看

{'体育': 0,
 '娱乐': 1,
 '家居': 2,
 '房产': 3,
 '教育': 4,
 '时尚': 5,
 '时政': 6,
 '游戏': 7,
 '科技': 8,
 '财经': 9}

In [8]:
df.category = df['category'].map(label_dict) #map做映射，把category 映射成number

In [9]:
df.head()

,category,text
0,0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1,0,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2,0,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3,0,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4,0,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...


In [10]:
from sklearn.model_selection import train_test_split #在数据集上随机划分出一定比例的训练集和测试集
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.category.values, 
                                                  test_size=0.1, 
                                                  random_state=42,
                                                  stratify=df.category.values)
#df.index.values：被划分的样本特征集。
#df.category.values：被划分的样本标签集。
#使用train_test_split拆分 训练集和测试集90%为训练集，随机数42

In [11]:
df['data_type'] = ['not_set']*df.shape[0]  #新建一列data_type

In [12]:
df.head()

,category,text,data_type
0,0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...,not_set
1,0,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...,not_set
2,0,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...,not_set
3,0,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...,not_set
4,0,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...,not_set


In [13]:
df.loc[X_train, 'data_type'] = 'train' #赋值data_type为训练集train
df.loc[X_val, 'data_type'] = 'val' #赋值data_type为测试集val

In [14]:
df.groupby(['category', 'data_type']).count()  #进行数据的分组统计类别，数据标签分布

text
category data_type      
0        train      4500
         val         500
1        train      4500
         val         500
2        train      4500
         val         500
3        train      4500
         val         500
4        train      4500
         val         500
5        train      4500
         val         500
6        train      4500
         val         500
7        train      4500
         val         500
8        train      4500
         val         500
9        train      4500
         val         500

In [15]:
from transformers import BertTokenizer #导入BertTokenizer分词器
from torch.utils.data import TensorDataset

In [16]:
# 中文分词
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-chinese'
)

In [17]:
max_length=256 #句子最大长度256

# 以batch形式编码句子,输入为句子的列表
encoded_data_train = tokenizer.batch_encode_plus(  #采用bert 分词器进行分词（训练集）
    df[df.data_type=='train'].text.values,  #文本值
    add_special_tokens=True,  #序列加上特殊符号，如[CLS],[SEP]
    return_attention_mask=True,  #返回
    padding='max_length', #补填到最大句子长度
    truncation=True,  #截断操作，超过则截取
)

encoded_data_val = tokenizer.batch_encode_plus(  #采用bert 分词器进行分词（测试集）
    df[df.data_type=='val'].text.values, #文本值
    add_special_tokens=True,  #序列加上特殊符号，如[CLS],[SEP]
    return_attention_mask=True,
    padding='max_length', #补填到最大句子长度
    truncation=True,   #截断操作，超过则截取
)

input_ids_train = encoded_data_train['input_ids']  #获取训练集bert输入（input_ids）
attention_masks_train = encoded_data_train['attention_mask']  #获取bert输入（attention_mask）
labels_train = torch.tensor(df[df.data_type=='train'].category.values) #获取label 并转化成tensor

input_ids_val = encoded_data_val['input_ids']  #获取测试集bert输入（input_ids）
attention_masks_val = encoded_data_val['attention_mask'] #获取bert输入（attention_mask）
labels_val = torch.tensor(df[df.data_type=='val'].category.values) #获取label 并转化成tensor

In [18]:
df[df.data_type=='train'] #查看训练数据

,category,text,data_type
0,0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...,train
1,0,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...,train
2,0,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...,train
3,0,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...,train
5,0,阿的江：八一需重新定位 我们有机会但该进的没进新浪体育讯12月24日，回到主场的北京金隅迎战...,train
...,...,...,...
49994,9,基金灰幕：1430亿沉没的红利赵娟2009年初，家住湖北宜昌的毛女士打开很久没有看过的基金账...,train
49995,9,打好投资的决胜局□国泰基金 阿邰就长期而言，资产配置占投资成功与否的决定因素高达90%以上。...,train
49996,9,昔明星基金今年首月负收益 QDII成今年胸闷基金□晨报记者 陈重博2009年以来，所有偏股型...,train
49998,9,沪基指半日涨2.01% 两市封基近乎全线上扬全景网2月6日讯 沪深基金指数周五早盘大幅收高，...,train


In [19]:
input_ids_train = torch.tensor(input_ids_train)  #训练集输入转换成tensor
input_ids_val = torch.tensor(input_ids_val) #验证集输入转换成tensor
attention_masks_train = torch.tensor(attention_masks_train) #训练集attention_masks转换成tensor
attention_masks_val = torch.tensor(attention_masks_val) #验证集attention_masks转换成tensor

dataset_train = TensorDataset(input_ids_train,   #采用TensorDataset包封装成训练集
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,  #采用TensorDataset包封装成测试集
                            attention_masks_val,
                           labels_val)

In [20]:
len(dataset_train) #查看训练集长度

45000

In [21]:
dataset_train.tensors  #查看训练集

(tensor([[ 101, 7716, 3236,  ..., 1086, 2612,  102],
         [ 101, 1555, 4448,  ..., 1546,  671,  102],
         [ 101, 1094, 1092,  ..., 1912, 5093,  102],
         ...,
         [ 101, 3212, 3209,  ..., 7025,  711,  102],
         [ 101, 3772, 1825,  ..., 2845,  122,  102],
         [ 101, 4281, 2399,  ..., 2141, 1213,  102]]),
 tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 tensor([0, 0, 0,  ..., 9, 9, 9]))

In [22]:
class BertCNN(nn.Module):  #定义BertCNN类 继承nn.Module
    """
    Bert+TextCNN2D
    """

    def __init__(self, num_filters, hidden_size, filter_sizes, dropout, num_classes,
                 trained=True):  #定义初始化参数

        super(BertCNN, self).__init__()  # 子类继承nn.Module

        self.bert = BertModel.from_pretrained('bert-base-chinese')  #定义预训练bert模型
        for param in self.bert.parameters():
            param.requires_grad = trained  #设置bert参数参与网络更新

        self.convs = nn.ModuleList([
            nn.Conv2d(
                in_channels=1,  #输入维度
                out_channels=num_filters,  #输出维度
                kernel_size=(k, hidden_size))  #卷积核大小
            for k in filter_sizes
        ])  #使用nn.ModuleList集成多个卷积层

        self.dropout = nn.Dropout(dropout)  #定义dropout层

        # 线性映射，（len(filter_sizes) * n_filters）（卷积核大小*卷积核数量）到num_classes
        self.fc_cnn = nn.Linear(num_filters * len(filter_sizes), num_classes)
#         self.act = nn.Softmax()

    def conv_and_pool(self, x, conv):
        """
        封装卷积和池化
        """

        conved = F.relu(conv(x)).squeeze(3)  #tensor先进入卷积层再经过relu激活并剪裁掉tensor的索引为3的维度信
        pooled = F.max_pool1d(conved, conved.size(2)).squeeze(2)  #进行max pooling 池化操作并剪裁掉tensor的索引为4的维度信息

        return pooled  #return 结果

    def forward(self, context, bert_masks, seq_len):
        # context [batch_size, sen_len]

        # 文本encode，文本cls
        encoder_out, _ = self.bert(context, attention_mask=bert_masks, return_dict=False)
        # encoder_out [batch_size, sen_len, H=768]
        # _ [batch_size, H=768]

        # encoder_out做dropout
        encoder_out = self.dropout(encoder_out)

        out = encoder_out.unsqueeze(1)  #添加tensor的索引为1的维度
        # out [batch_size, 1, sen_len, H=768]

        # 卷积池化，再拼接
        out = torch.cat([
            self.conv_and_pool(out, conv)
            for conv in self.convs
        ], dim=1)
        # out == [batch_size, n_filters * len(filter_sizes)]

        out = self.dropout(out)
        
        out = self.fc_cnn(out)
        
        return out

In [23]:
num_filters = 100   #定义卷积核输出维度
hidden_size = 768   #定义隐藏层维度
filter_sizes = [3,4,5]  #定义卷积核大小
dropout = 0.4   #定义dropout系数
num_classes = len(label_dict)  #定义几分类

model = BertCNN(num_filters,hidden_size,filter_sizes,dropout,num_classes)  #实例化模型

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 2  #定义batch_size

dataloader_train = DataLoader(  #使用DataLoader封装使模型能进行batch训练
    dataset_train,   #训练集
    sampler=RandomSampler(dataset_train),  #随机采样
    batch_size=batch_size,  #定义batch_size
    drop_last=True  #丢弃不满足一个batch的数据
)

dataloader_val = DataLoader(  #使用DataLoader封装使模型能进行batch训练
    dataset_val,  #测试集
    sampler=RandomSampler(dataset_val),  #随机采样
    batch_size=batch_size,  #定义batch_size
    drop_last=True  #丢弃不满足一个batch的数据
)

In [25]:
from transformers import AdamW, get_linear_schedule_with_warmup

criterion = nn.CrossEntropyLoss()  #定义loss函数
optimizer = torch.optim.AdamW(   #定义优化器
    model.parameters(),
    lr = 1e-5,  #学习率
    eps = 1e-8  #AdamW系数
)

In [26]:
epochs = 8  #定义训练epochs

scheduler = get_linear_schedule_with_warmup(  #进行学习率warmup
    optimizer,  #优化器
    num_warmup_steps=0,   #warmup起点为0
    num_training_steps = len(dataloader_train)*epochs
)  #训练轮数

In [27]:
import numpy as np
from sklearn.metrics import f1_score

In [28]:
def f1_score_func(preds, labels):  #计算f1_score函数
    preds_flat = np.argmax(preds, axis=1).flatten()  #获取预测label
    labels_flat = labels.flatten()  #获取真实label
    return f1_score(labels_flat, preds_flat, average = 'weighted')  #计算f1_score使用‘weighted’权重

In [29]:
def accuracy_per_class(preds, labels):  #对每个类别进行计算准确率
    label_dict_inverse = {v: k for k, v in label_dict.items()}  #定义索引与类别之间的映射字典
    
    preds_flat = np.argmax(preds, axis=1).flatten()  #获取预测label
    labels_flat = labels.flatten()  #获取真实label
    
    for label in np.unique(labels_flat):  #对于每一类
        y_preds = preds_flat[labels_flat==label]  #获取预测label
        y_true = labels_flat[labels_flat==label]  #获取真实label
        print(f'Class: {label_dict_inverse[label]}')  #打印类别信息
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')  #打印准确率

In [30]:
import random

seed_val = 17  #设置随机数
random.seed(seed_val)  #设置随机种子
np.random.seed(seed_val)  #设置np随机种子
torch.manual_seed(seed_val)  #设置torch随机种子
torch.cuda.manual_seed_all(seed_val)  #设置cuda随机种子

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  #设置训练的设备如有gpu选择gpu没有就cpu
model.to(device)  #将模型放入 device
print(device)  #打印 device

cuda


In [32]:
# 自定义bert
def evaluate(dataloader_val):  #定义评估函数

    model.eval()  #模型做为预测
#     model = model.to(device)
    loss_val_total = 0  #loss_val_total置零
    predictions, true_vals = [], []  #定义预测和真实值
    
    for batch in tqdm(dataloader_val):  #对于每个batch
        
        batch = tuple(b.to(device) for b in batch)  #batch放入device
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }   #获取模型输入

        with torch.no_grad():         #不做梯度求导    
            outputs = model(inputs.get('input_ids'), inputs.get('attention_mask'), 256).squeeze(1)
             #获取模型输出并裁减索引位置为1的的维度
        loss = criterion(outputs, inputs['labels'])  #计算loss
        logits = outputs  #获取输出
        loss_val_total += loss.item()  #累计loss

        logits = logits.detach().cpu().numpy()  #将logits放回cpu并转成numpy
        label_ids = inputs['labels'].cpu().numpy()  #将labels放回cpu并转成numpy
        predictions.append(logits)  #预测值累加
        true_vals.append(label_ids)  #真实值累加
    
    loss_val_avg = loss_val_total/len(dataloader_val)   #计算平均loss
    
    predictions = np.concatenate(predictions, axis=0)  #合并所有预测值
    true_vals = np.concatenate(true_vals, axis=0)   #合并所有真实值
            
    return loss_val_avg, predictions, true_vals   #返回结果

In [33]:
# 自定义bert
for epoch in tqdm(range(1, epochs+1)):  #对于每一轮
# for epoch in tqdm(range(1, 1)):
    true_vals = []  #定义真实值列表
    pred_label = []  #定义预测值列表
    model.train()  #模型进行训练
#     model = model.to(device)
    loss_train_total = 0  #初始化loss数值
    
    progress_bar = tqdm(dataloader_train,   #定义一个可视化的训练进度条
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:  #对于每个batch
        model.zero_grad()  #清空梯度信息
        batch = tuple(b.to(device) for b in batch)  #batch放入device

        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(inputs.get('input_ids'), inputs.get('attention_mask'), 256)
        #获取模型输出


        loss = criterion(outputs, inputs['labels'])  #计算loss
        loss_train_total +=loss.item()  #累计loss
        loss.backward()  #反向传播
        
        
        
        label_ids = inputs['labels'].cpu().numpy()  #将labels放回cpu并转成numpy
        true_vals.append(label_ids)  #累加 labels 
        pred_y = np.argmax(outputs.cpu().detach().numpy(), axis=1).flatten()  #将预测数值放回cpu并转成numpy
        pred_label.append(pred_y)  #累加 outputs
        
        
#         true_vals = true_vals.flatten()
        

        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #梯度裁减
        
        optimizer.step()  #优化器迭代
        scheduler.step()   #学习率迭代
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})       #可视化loss数值
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch：{epoch}')  #可视化步数
    #     tqdm.write(f'socre: {report}')
    
    val_loss, predictions, true_vals_val = evaluate(dataloader_val) #模型验证
    val_f1 = f1_score_func(predictions, true_vals_val) #计算f1
#     val_acc = accuracy_per_class(predictions, true_vals)

    preds_val = np.argmax(predictions, axis=1).flatten() #获取预测数值
    report_val = classification_report(true_vals_val, preds_val, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], target_names=['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经'])
    print(report_val) #采用classification_report输出可视化验证集评估指标并打印
    
    tqdm.write(f'Validation loss: {val_loss}') #可视化loss
    tqdm.write(f'F1 Score (weighted): {val_f1}') #可视化f1
#     tqdm.write(f'Accuracy (weighted): {val_acc}')
  

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/22500 [00:00<?, ?it/s]


Epoch：1


  0%|          | 0/2500 [00:00<?, ?it/s]

              precision    recall  f1-score   support

          体育       0.99      1.00      0.99       500
          娱乐       1.00      1.00      1.00       500
          家居       0.99      0.98      0.99       500
          房产       0.96      0.94      0.95       500
          教育       0.97      0.96      0.96       500
          时尚       0.99      0.98      0.98       500
          时政       0.94      0.97      0.96       500
          游戏       0.99      0.99      0.99       500
          科技       0.96      0.95      0.96       500
          财经       0.94      0.96      0.95       500

    accuracy                           0.97      5000
   macro avg       0.97      0.97      0.97      5000
weighted avg       0.97      0.97      0.97      5000

Validation loss: 0.12269909604619024
F1 Score (weighted): 0.9728313555954775


...

              precision    recall  f1-score   support

          体育       1.00      1.00      1.00       500
          娱乐       0.99      1.00      1.00       500
          家居       0.99      0.99      0.99       500
          房产       0.96      0.97      0.97       500
          教育       0.99      0.97      0.98       500
          时尚       0.99      1.00      0.99       500
          时政       0.98      0.97      0.97       500
          游戏       0.99      0.99      0.99       500
          科技       0.97      0.97      0.97       500
          财经       0.96      0.96      0.96       500

    accuracy                           0.98      5000
   macro avg       0.98      0.98      0.98      5000
weighted avg       0.98      0.98      0.98      5000

Validation loss: 0.16158773748300423
F1 Score (weighted): 0.9817862518689208


Epoch 8:   0%|          | 0/22500 [00:00<?, ?it/s]


Epoch：8


  0%|          | 0/2500 [00:00<?, ?it/s]

              precision    recall  f1-score   support

          体育       1.00      1.00      1.00       500
          娱乐       1.00      1.00      1.00       500
          家居       0.99      0.99      0.99       500
          房产       0.97      0.96      0.97       500
          教育       0.98      0.97      0.97       500
          时尚       1.00      0.99      0.99       500
          时政       0.97      0.98      0.98       500
          游戏       0.99      0.99      0.99       500
          科技       0.97      0.98      0.97       500
          财经       0.96      0.95      0.96       500

    accuracy                           0.98      5000
   macro avg       0.98      0.98      0.98      5000
weighted avg       0.98      0.98      0.98      5000

Validation loss: 0.15992317895119781
F1 Score (weighted): 0.9823828769577604
